In [23]:
#Importando as bibliotecas
import arcpy
from arcpy import env
import os
from arcpy.ia import *
env.overwriteOutput = True

In [2]:
#Diretórios
#Primeira coisa é criar o workspace do GDB para quando for exportar algo já adicionar nele
workspace1 = r"E:\Cursos\GeoAI\GeoAI_02\GeoAI_02\GeoAI_02.gdb"
#Uma pasta dentro do diretório que estou trabalhando
workspace2 = r"E:\Cursos\GeoAI\GeoAI_02\GeoAI_02"

In [3]:
#Diretório com as imagens
imagem_dir = r"E:\Cursos\GeoAI\GeoAI_02\Imagens"

#Bandas para tratar

banda0 = os.path.join(imagem_dir,'CBERS_4A_WPM_20210727_211_115_L4_BAND0.tif')
banda1 = os.path.join(imagem_dir,'CBERS_4A_WPM_20210727_211_115_L4_BAND1.tif')
banda2 = os.path.join(imagem_dir,'CBERS_4A_WPM_20210727_211_115_L4_BAND2.tif')
banda3 = os.path.join(imagem_dir,'CBERS_4A_WPM_20210727_211_115_L4_BAND3.tif')
banda4 = os.path.join(imagem_dir,'CBERS_4A_WPM_20210727_211_115_L4_BAND4.tif')

Eliminar Ruído

In [4]:
#Diretório do filtro de ruído
focal1 = os.path.join(workspace2, "focal_lost_banda3.tif")

In [5]:
#Realizar a correção de Ruído
with arcpy.EnvManager(scratchWorkspace=workspace2, workspace=workspace2):
    out_raster = arcpy.ia.FocalStatistics(banda3, "Rectangle 3 3 CELL","MEAN","DATA", 90); out_raster.save(focal1)

Corrigir erro geométrico

In [31]:
#Realizar a correção geométrica
arcpy.management.RegisterRaster(banda1,
                               "REGISTER",
                               "World Imagery",
                               None,
                               'POLYSIMILARITY',
                               None,
                               None)


<Result ''>

In [33]:
arcpy.management.RegisterRaster(banda0,
                               "REGISTER",
                               banda1,
                               None,
                               'POLYSIMILARITY',
                               None,
                               None)

<Result ''>

In [34]:
arcpy.management.RegisterRaster(banda2,
                               "REGISTER",
                               banda1,
                               None,
                               'POLYSIMILARITY',
                               None,
                               None)

<Result ''>

In [35]:
arcpy.management.RegisterRaster(banda3,
                               "REGISTER",
                               banda1,
                               None,
                               'POLYSIMILARITY',
                               None,
                               None)

<Result ''>

In [36]:
arcpy.management.RegisterRaster(banda4,
                               "REGISTER",
                               banda1,
                               None,
                               'POLYSIMILARITY',
                               None,
                               None)

<Result ''>

Corrigir o Histograma

In [13]:
#Calcular estatísticas
arcpy.management.CalculateStatistics(banda3, 1, 1, [], "OVERWRITE", r'in_memory\feature')

<Result 'E:\\Cursos\\GeoAI\\GeoAI_02\\Imagens\\CBERS_4A_WPM_20210727_211_115_L4_BAND3.tif'>

In [7]:
#Diretório de Histogramas
histnew = os.path.join(workspace2, "new_hist.tif")

#Realizar realce histograma
StatisticsHistogram_raster = arcpy.ia.StatisticsHistogram(banda3, [61,214,121.96,1.0], None)
StatisticsHistogram_raster.save(histnew)

Realizar operação entre bandas

In [37]:
#Realizar operação aritmética

calculate = os.path.join(workspace2, 'NDVI.tif')

calculate_rastes = RasterCalculator([banda4,banda1], ['x','y'], '(x-y)/(x+y)')
calculate_rastes.save(calculate)

Realizar a composição das bandas

In [38]:
#Diretório da composição

composicao_output = os.path.join(workspace2, 'composicao_output.tif')

In [39]:
#Bandas para composição
composicao = r'{};{};{}'.format(banda3,banda2,banda1)

#Realizar a composição das bandas
arcpy.management.CompositeBands(composicao, composicao_output)

<Result 'E:\\Cursos\\GeoAI\\GeoAI_02\\GeoAI_02\\composicao_output.tif'>

Transformar as cores

In [40]:
#Diretório das operação aritmética

conver_color = os.path.join(workspace2, 'composicao_hsv_3bands.tif')

In [41]:
#Realizar a transformação de cores
out_hsv_3bands = ColorspaceConversion(composicao_output,'rgb_to_hsv')
out_hsv_3bands.save(conver_color)

Realizar a fusão de bandas

In [42]:
#Diretório do Pansharnepe
pan_output = os.path.join(workspace2, 'Pansharpene.tif')

#Realizar o pansharpene
arcpy.management.CreatePansharpenedRasterDataset('composicao_output.tif',
                                                1,2,3,4,
                                                pan_output,
                                                banda0,
                                                'Gram-Schmidt',
                                                0.166,0.167,0.167, 0.5,
                                                'UNKNOWN')

<Result 'E:\\Cursos\\GeoAI\\GeoAI_02\\GeoAI_02\\Pansharpene.tif'>

Realizar Filtragem

* Passa baixa - Ressalta as estruturas maiores na imagem;
* Passa alta - Ressalta as estruturas menores na imagem.

In [43]:
#Diretório do aquivo filtro
filter_output = os.path.join(workspace2,'filter_banda3_PB.tif')

#Realizar o filtro passa-baixa
with arcpy.EnvManager(scratchWorkspace=workspace1, workspace=workspace1):
    out_raster = arcpy.sa.Filter(banda3, 'LOW', 'DATA'); out_raster.save(filter_output)

In [44]:
#Diretório do aquivo filtro
filter_output = os.path.join(workspace2,'filter_banda3_PA.tif')

#Realizar o filtro passa-alta
with arcpy.EnvManager(scratchWorkspace=workspace1, workspace=workspace1):
    out_raster = arcpy.sa.Filter(banda3, 'HIGH', 'DATA'); out_raster.save(filter_output)

In [45]:
#Diretório do aquivo convulacional
filter_output = os.path.join(workspace2,'laplaciano.tif')

#Realizar o filtro passa-baixa
laplaciano = arcpy.ia.Convolution(banda3, 16)
laplaciano.save(filter_output)